In [23]:
import pandas as pd
import random
import os
import numpy as np
import pickle

# load word vecs

In [2]:
data_path = r"C:\Users\alexa\Documents\data\wir\project"
fname = r"cc.en.300.bin"

In [3]:
%%time
import fasttext
model = fasttext.load_model(
    os.path.join(data_path, fname))

Wall time: 1min 31s


In [14]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

In [15]:
data_path = r"C:\Users\alexa\Documents\data\wir\project"
fname = r"wiki-news-300d-1M-subword.vec"

In [ ]:
%time
wv = load_vectors(
    os.path.join(data_path, fname))

In [4]:
import csv

In [5]:
%%time
data_path = r"C:\Users\alexa\Documents\data\nlp\nlp_hw3"
glove_data_file = 'glove.6B.300d.txt'
emb = pd.read_table(os.path.join(data_path, glove_data_file), sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)

Wall time: 52.9 s


# generate img embeddings

In [4]:
from imgsearch import cosine_sim

def getwv(s):
    return model.get_word_vector(s)

def getwv_multi_labels(labels):
    sublabels = labels.split(', ')
    return sum([getwv(subl) for subl in  sublabels]) / len(sublabels)

In [5]:
get_weighted_sum_vec = lambda tup: sum([getwv(label) * weight for weight, label in tup])
get_weighted_sum_vec_multilbl = lambda tup: sum([getwv_multi_labels(label) * weight for weight, label in tup])

get_vec_first_label = lambda tup: sum([getwv_multi_labels(label) for weight, label in [tup[0]]])

In [7]:
path='static/img_top10_labels_w_weights.csv'
    
# load image label data
labelled_df = pd.read_csv(path)

In [14]:
from ast import literal_eval
labelled_df.content = labelled_df.content.apply(literal_eval)

In [174]:
%%time
img_emb_dict = dict(zip(
    labelled_df.img_id,
    labelled_df.content.apply(get_weight_vec_multilbl)
    #labelled_df.content.apply(get_vec_first_label)
))

Wall time: 935 ms


In [175]:
import pickle
#with open(r"static/img_emb_dict.pkl", "wb") as f:
#    pickle.dump(img_emb_dict, f)

# build search routine

In [176]:
#with open(r"static/img_emb_dict.pkl", "rb") as f:
#    img_emb_dict = pickle.load(f)

In [177]:
def image_search_res5(query, img_emb_dict):
    
    # get query embedding representation
    q = getwv_multi_labels(query)
    
    if q.sum() != 0:
        
        # parse query
        # ...
        
        # map image ids to names
        img_id_to_name = dict(zip(labelled_df.img_id, labelled_df.image_name))
        
        # score and rank images
        img_idxs = img_emb_dict.keys()
        score = [(
            img_id_to_name[key]
            ,cosine_sim(q, img_emb_dict[key])
        ) for key in img_idxs]
        ranked = sorted(score, key=lambda tup: tup[1], reverse=True)
    
        # output list of images and proba
        return ranked[:12]
    else:
        return []

In [181]:
query = 'hiking shoes'
id_ = 14522
print(
    labelled_df.query("img_id == '{}'".format(id_)).image_name.values[0])
print(
    cosine_sim(img_emb_dict['{}'.format(id_)], getwv_multi_labels(query)))
print(
    labelled_df.query("img_id == '{}'".format(id_)).loc[:,'content'].values[0][:3])

1828-14522-large_default-sac-a-dos-rucksack-rains.jpg
0.15093912
((0.18596436083316806, 'clog, geta, patten, sabot'), (0.17596831917762756, 'mailbag, postbag'), (0.151167631149292, 'sandal'))


In [179]:
query = 'shoes'
image_list = image_search_res5(query, img_emb_dict)
with open(r"static/image_list.pkl", "wb") as f:
    pickle.dump(image_list, f)
image_list

[('1588-13961-large_default-chaussons-en-laine-naturelle.jpg', 0.60665333),
 ('912-8278-large_default-flasque-en-cuivre.jpg', 0.58782387),
 ('1057-13488-large_default-t-shirt-to-the-sea.jpg', 0.5869558),
 ('1588-13261-large_default-chaussons-en-laine-naturelle.jpg', 0.58575124),
 ('1828-14527-large_default-sac-a-dos-rucksack-rains.jpg', 0.57823175),
 ('1828-14522-large_default-sac-a-dos-rucksack-rains.jpg', 0.57441026),
 ('1191-10092-large_default-maillot-de-bain-iguazu.jpg', 0.54713625),
 ('1588-13960-large_default-chaussons-en-laine-naturelle.jpg', 0.54574186),
 ('1828-14524-large_default-sac-a-dos-rucksack-rains.jpg', 0.5187537),
 ('1257-10737-large_default-bracelet-manille-argent-massif.jpg', 0.5122339),
 ('1755-14207-large_default-echarpe-alphonse.jpg', 0.51223165),
 ('1783-14379-large_default-affiches-des-pays-du-monde.jpg', 0.51004076)]

# add weights to image labels

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from imgsearch import cosine_sim
import numpy as np

In [9]:
#pd.read_csv('static/imagenet_classes_to_labels_map.txt')
with open('static/imagenet_classes_to_labels_map.txt') as f:
    labels = eval(f.read())

In [10]:
def get_topn_labels(row, n=10):
    outdf = row.astype(float).nlargest(n).reset_index()
    outdf.columns = ['label_id','percentage']
    outdf.label_id = outdf.label_id.astype(int)
    outdf['label_name'] = outdf.label_id.map(labels)
    return outdf

In [11]:
%%time

n_img = 10000

# get weight data
data_path = r"C:\Users\alexa\Documents\data\wir\project\img_proba_vecs_vgg19.csv"
dfw = pd.read_csv(data_path, low_memory=False)

# generate df with descr of top10 classes
labelled_df = dfw.loc[:, dfw.columns[:3]].copy()
labelled_df['content'] = ''

for i, image_name in enumerate(dfw.image_name.loc[:n_img]):
    topnlabels = get_topn_labels(dfw.loc[i, dfw.columns[3:]])
    labelled_df.loc[i, 'content'] = tuple(zip(topnlabels.percentage, topnlabels.label_name)) # keep weight info

# select top n images
labelled_df = labelled_df.loc[:n_img].copy()

Wall time: 1min 11s


In [12]:
    path='static/img_top10_labels_w_weights.csv'
    
    # load image label data
    labelled_df.to_csv(path, index=False)